# SQL

Был преобретен крупный сервис для чтения книг по подписке. Имеется база данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Цель: сформулировать ценностное предложение для нового продукта

Задача: проанализировать базу данных:
    - посчитать количество книг, вышедших после 1 января 2000 года;
    - для каждой книги посчитайте количество обзоров и среднюю оценку;
    - определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
    - определить автора с самой высокой средней оценкой книг;
    - посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
    
Ход исследования:
    1. Загрузка данных
    2. Исследование данных
    3. Выводы

# Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
#query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

## Изучение данных

Выведем первые пять строк каждого датасета

In [2]:
def data_descr(data):
    query = '''
    SELECT *
    FROM '''+data+'''
    LIMIT 5 '''
    
    print(f'Вывод первых 5 строк датасета {data}:')
    display(pd.io.sql.read_sql(sql=text(query), con = con))
    print('='*50)

In [3]:
data = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for i in data:
    data_descr(i)

Вывод первых 5 строк датасета books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Вывод первых 5 строк датасета authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Вывод первых 5 строк датасета publishers:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Вывод первых 5 строк датасета ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Вывод первых 5 строк датасета reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [4]:
query_NB = '''
    SELECT *
    FROM books
    WHERE book_id IS NULL
    OR title IS NULL 
    OR num_pages IS NULL 
    OR publication_date IS NULL
    OR publisher_id IS NULL'''
    
pd.io.sql.read_sql(sql=text(query_NB), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id


In [5]:
query_NA = '''
    SELECT *
    FROM authors
    WHERE author_id IS NULL
    OR author IS NULL

    '''
    
pd.io.sql.read_sql(sql=text(query_NA), con = con)

,author_id,author


In [6]:
query_NP = '''
    SELECT *
    FROM publishers
    WHERE publisher IS NULL
    OR publisher_id IS NULL

    '''
    
pd.io.sql.read_sql(sql=text(query_NP), con = con)

,publisher_id,publisher


In [7]:
query_NR = '''
    SELECT *
    FROM ratings
    WHERE rating_id IS NULL
    OR book_id IS NULL
    OR rating IS NULL
    OR username IS NULL
    '''

pd.io.sql.read_sql(sql=text(query_NR), con = con)

,rating_id,book_id,username,rating


In [8]:
query_NRW = '''
    SELECT *
    FROM reviews
    WHERE review_id IS NULL
    OR book_id IS NULL
    OR text IS NULL
    OR username IS NULL
    '''

pd.io.sql.read_sql(sql=text(query_NRW), con = con)

,review_id,book_id,username,text


Проверим еще на возможные дубликаты таблицы books. autors, publishers.
Пользователей проверить корректно у нас не выйдет: есть полные тезки, пользователи могут некорректно заполнять данные (а-ля Иванов Иван Ивановочич).

В данном исследовании нам интереснее будет их активность.

In [9]:
query_DA = '''
    SELECT author, COUNT(author)
    FROM authors
    GROUP BY author
    HAVING COUNT(author)>1
    '''

pd.io.sql.read_sql(sql=text(query_DA), con = con)

,author,count


In [10]:
query_DP = '''
    SELECT publisher, COUNT(publisher)
    FROM publishers
    GROUP BY publisher
    HAVING COUNT(publisher)>1
    '''

pd.io.sql.read_sql(sql=text(query_DP), con = con)

,publisher,count


In [11]:
query_DB = '''
    SELECT title, COUNT(title)
    FROM books
    GROUP BY title
    HAVING COUNT(title)>1
    '''

pd.io.sql.read_sql(sql=text(query_DB), con = con)

,title,count
0,Memoirs of a Geisha,2


In [12]:
query_DB1 = '''
    SELECT *
    FROM books
    WHERE title LIKE 'Memoirs of a Geisha'
    '''

pd.io.sql.read_sql(sql=text(query_DB1), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Мемуары гейши были опубликованы 2 разными издательствами в один год. 

**Вывод**

База данных состоит из 5 датасетов: 'books', 'authors', 'publishers', 'ratings', 'reviews'.
Пропусков и дубликатов нет.



## Количество книг, вышедших после 1 января 2000 года

In [13]:
query1 = '''
            SELECT COUNT(DISTINCT book_id)
            FROM books
            WHERE publication_date >'2000-01-01'
        '''
pd.io.sql.read_sql(sql=text(query1), con = con)


,count
0,819


База данных содержит **819** книг, которые вышли после 1.01.2020 годаа

## Расчет количества обзоров и средней оценки для каждой книги.

In [14]:
query2 = '''
    SELECT b.book_id,
           b.title,
           ROUND(AVG(r.rating),1) AS avg_rating,
           COUNT(DISTINCT review_id) AS count_view
    FROM books AS b
    LEFT JOIN ratings AS r ON b.book_id =r.book_id
    LEFT JOIN reviews AS rev ON b.book_id=rev.book_id
    GROUP BY b.book_id
    ORDER BY count_view DESC,
             avg_rating DESC
             '''

pd.read_sql_query(sql=text(query2), con = con)


,book_id,title,avg_rating,count_view
0,948,Twilight (Twilight #1),3.7,7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.4,6
2,656,The Book Thief,4.3,6
3,299,Harry Potter and the Chamber of Secrets (Harry...,4.3,6
4,734,The Glass Castle,4.2,6
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,4.0,0
996,387,Leonardo's Notebooks,4.0,0
997,221,Essential Tales and Poems,4.0,0
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.7,0


## Издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [15]:
query3 = '''
    SELECT p.publisher,
           COUNT(DISTINCT b.book_id) AS count_books
    FROM books AS b
    JOIN publishers AS p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY count_books DESC
    LIMIT 1
    '''

pd.read_sql_query(sql=text(query3), con = con)

,publisher,count_books
0,Penguin Books,42


**Penguin Books** выпустило наибольшее число книг.

## Автор с самой высокой средней оценкой книг

In [16]:
query4 = '''
    SELECT a.author,
    ROUND(AVG(br.avg_rating),3) AS avg_all
    FROM books AS b
    JOIN authors AS a ON b.author_id = a.author_id
    JOIN (
        SELECT book_id,
        COUNT(rating_id) AS count_rat,
        AVG(rating) AS avg_rating
        FROM ratings
        GROUP BY book_id) AS br ON b.book_id = br.book_id
    WHERE count_rat >= 50
    GROUP BY author
    ORDER BY avg_all DESC
    LIMIT 1
    '''

pd.read_sql_query(sql=text(query4), con = con)

,author,avg_all
0,J.K. Rowling/Mary GrandPré,4.284


Самую высокую оценку получила автор J.K. Rowling/Mary GrandPré.
Успешная колобарация знаменитой писательницы и талантливого иллюстратора :)

## Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [17]:
query5 = '''
    WITH rt AS (
        SELECT username,
            COUNT(rating_id) AS count_rating
            FROM ratings
            GROUP BY ratings.username
            HAVING COUNT(rating_id) > 48)
    SELECT AVG(count_rev)
    FROM rt
    JOIN (SELECT reviews.username,
              COUNT(review_id) AS count_rev
              FROM reviews
              GROUP BY reviews.username) AS rew ON rew.username= rt.username
    
    '''
pd.read_sql_query(sql=text(query5), con = con)

,avg
0,24.0


Пользователи, которые поставили более 48 оценок, в среднем оставляют 24 обзора.

## Выводы

Была Загружена база данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Все датасеты были изучены, проверны на пропуски и дубликаты в данных.

Был проведен исследовательский анализ:
 - Количество книг, вышедших после 1 января 2000 года - 819.
 - Был получен расчет количества обзоров и средней оценки для каждой книги.
 - Издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books
 - Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.
 - Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.